In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from keras.applications import DenseNet121

# Load and preprocess data


In [2]:
train_dir = 'OCT2017_/train'
test_dir = 'OCT2017_/test'
val_dir='OCT2017_/val'

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=(224, 224),
                                               batch_size=32,
                                               class_mode='categorical')

test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=(224, 224),
                                             batch_size=32,
                                             class_mode='categorical')
val_data = val_datagen.flow_from_directory(val_dir,
                                             target_size=(224, 224),
                                             batch_size=32,
                                             class_mode='categorical')
# Build and compile the model


Found 83484 images belonging to 4 classes.
Found 968 images belonging to 4 classes.
Found 32 images belonging to 4 classes.


In [3]:
model = Sequential()

densenet = DenseNet121(include_top=False, input_shape=(224, 224, 3))

model.add(densenet)
model.add(Flatten())
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

model.compile(optimizer=Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)



C:\Users\DSL-187\AppData\Roaming\Python\Python38\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [4]:
history = model.fit(train_data,
                    steps_per_epoch=train_data.n//train_data.batch_size,
                    epochs=20,
                    validation_data=val_data,
                    validation_steps=val_data.n//val_data.batch_size,
                    callbacks=[reduce_lr])

# Evaluate the model



Epoch 1/20
2608/2608 [==============================] - 58761s 23s/step - loss: 0.2768 - accuracy: 0.9099 - val_loss: 0.0175 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 2/20
2608/2608 [==============================] - 56124s 22s/step - loss: 0.2028 - accuracy: 0.9333 - val_loss: 0.0306 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 3/20
2608/2608 [==============================] - 39847s 15s/step - loss: 0.1814 - accuracy: 0.9398 - val_loss: 0.0274 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 4/20
2608/2608 [==============================] - 39865s 15s/step - loss: 0.1680 - accuracy: 0.9444 - val_loss: 0.0590 - val_accuracy: 0.9688 - lr: 0.0010
Epoch 5/20
2608/2608 [==============================] - 39879s 15s/step - loss: 0.1201 - accuracy: 0.9595 - val_loss: 0.0077 - val_accuracy: 1.0000 - lr: 2.0000e-04
Epoch 6/20
2608/2608 [==============================] - 38376s 15s/step - loss: 0.1087 - accuracy: 0.9630 - val_loss: 0.0039 - val_accuracy: 1.0000 - lr: 2.0000e-04
Epoch 7/20
2608/2608 [====

In [ ]:
test_loss, test_acc = model.evaluate(test_data)

print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

# Predict on new data
test_preds = model.predict(test_data)

# Convert predictions to labels
test_labels = test_data.classes
pred_labels = np.argmax(test_preds, axis=1)

# Generate confusion matrix
cm = confusion_matrix(test_labels, pred_labels)
cm_df = pd.DataFrame(cm, index=['CNV', 'DME', 'Drusen', 'Normal'], columns=['CNV', 'DME', 'Drusen', 'Normal'])
sns.heatmap(cm_df, annot=True)
plt.show()